In [ ]:
import sqlite3
from langgraph.graph import StateGraph, START, END
from langchain.chat_models import init_chat_model
from langgraph.graph.message import MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.types import interrupt, Command
import dotenv

dotenv.load_dotenv()

llm = init_chat_model("openai:gpt-4o-mini")

conn = sqlite3.connect(
    "memory.db",
    check_same_thread=False,
)

config = {
    "configurable" : {
        "thread_id": "3"
    }
}

In [ ]:
class State(MessagesState):
    custom_stuff : str

graph_builder = StateGraph(State)

In [ ]:
@tool
def get_human_feedback(poem: str):
    """
    Asks the user for feedback on the poem.
    Use this before returning the final response.
    """
    feedback = interrupt(f"Here is the poem, tell me what you think\n{poem}")
    return feedback

tools = [get_human_feedback]
tools_by_name = {tool.name : tool for tool in tools}
llm_with_tools = llm.bind_tools(tools=tools)

def chatbot(state: State):
    response = llm_with_tools.invoke(
        f"""
        You are an expert in maiking poems.
        User the `get_human_feedback` tool to get feedback on your poem.
        Only after you receive positive feedvack you can return the final poem.
        ALWAYS ASK FOR FEEDBACK FIRST.
        Here is  the conversation history:
        {state["messages"]}
        """
    )
    return {"messages":[response]}


In [ ]:
from multiprocessing import context
from typing import Literal
from langchain_core.messages import ToolMessage
from langgraph.graph import StateGraph, START, END

def tool_node(state: dict):
    result = []
    for tool_call in state["messages"][-1].tool_calls:
        tool = tools_by_name[tool_call["name"]]
        observation = tool.invoke(tool_call["args"])
        result.append(ToolMessage(content=observation, tool_call_id=tool_call["id"]))

    return {"messages":result}

def should_continue(state: MessagesState) -> Literal["tools", END]:

    messages = state["messages"]
    last_message = messages[-1]

    if last_message.tool_calls:
        return "tools"
    
    return END


In [ ]:
from subprocess import check_output


graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)

graph_builder.add_edge(START, "chatbot")
graph_builder.add_conditional_edges("chatbot", should_continue, ["tools", END] )
graph_builder.add_edge("tools", "chatbot")

graph = graph_builder.compile(
    checkpointer=SqliteSaver(conn)
)



In [ ]:
result = graph.invoke(
    {
        "messages" : [
            {"role":"user", "content":"Please make a poem about Python code."}
        ]
    },
    config = config,
)

In [ ]:
for message in result["messages"]:
    message.pretty_print()

In [ ]:
snapshot = graph.get_state(config)
snapshot.next

In [ ]:
response = Command(resume="It's too long!")

result = graph.invoke(response, config=config)

for message in result["messages"]:
    message.pretty_print()

In [ ]:
snapshot = graph.get_state(config)
snapshot.next

In [ ]:
response = Command(resume="It's good!")

result = graph.invoke(response, config=config)

for message in result["messages"]:
    message.pretty_print()

In [ ]:
snapshot = graph.get_state(config)
snapshot.next